In [1]:
import os

import numpy as np
import pandas as pd
import xgboost as xgb
from rich import print
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    roc_auc_score
)
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline


pd.set_option('display.max_rows', 500)

In [2]:
data_dir = "./data"

data_frames = []

for file_name in os.listdir(data_dir):
    if file_name.endswith(".csv"):
        df = pd.read_csv(os.path.join(data_dir, file_name), sep=";", decimal=",", na_values="NA")
        if "Unnamed: 0" in df.columns:
            df = df.drop(columns=["Unnamed: 0"])

        parts = file_name.split("_")
        bankruptcy_status = 0 if "nonbankrupt" in parts[0] else 1
        industry = parts[1]

        df["industry_agriculture"] = 1 if industry == "agriculture" else 0
        df["industry_construction"] = 1 if industry == "construction" else 0
        df["industry_manufacture"] = 1 if industry == "manufacture" else 0
        df["industry_retail"] = 1 if industry == "retail" else 0
        df["bankruptcy_label"] = bankruptcy_status

        data_frames.append(df)

combined_df = pd.concat(data_frames, ignore_index=True)


In [3]:
print(combined_df.describe())

V1            V2            V3            V4            V5  \
count   51095.000000  5.103200e+04  4.820800e+04  49158.000000  49247.000000   
mean       -7.363870 -2.021102e+01  7.121501e+01      9.467094     12.232457   
std       752.489636  1.246801e+04  2.649772e+04    275.715999    305.820606   
min   -146527.270000 -1.975000e+06 -6.989966e+05  -8911.000000  -8911.000000   
25%        -0.500000  0.000000e+00 -8.000000e-02      0.070000      0.540000   
50%         2.040000  9.390000e+00  1.670000e+00      0.300000      1.040000   
75%         9.810000  3.707000e+01  6.840000e+00      1.120000      2.280000   
max     18725.000000  1.735600e+06  5.707100e+06  26201.000000  27699.000000   

                 V6            V7            V8            V9           V10  \
count  49202.000000  4.843100e+04  4.566800e+04  4.798800e+04  3.837800e+04   
mean      13.262163  5.450944e+03  2.010796e+03  2.409203e+03  2.334857e+02   
std      318.808668  3.167082e+05  1.720157e+05  1.670290e+05  1.076977e+04   
min    -8911.000000 -7.602669e+04 -1.463262e+04 -1.281545e+05 -2.946954e+04   
25%        0.910000  1.371900e+02  2.495750e+01  4.231000e+01  7.530000e+00   
50%        1.390000  2.350700e+02  5.443500e+01  8.562000e+01  3.120000e+01   
75%        2.990000  4.509700e+02  1.051725e+02  1.759450e+02  8.473750e+01   
max    33200.000000  4.302540e+07  2.603653e+07  2.255448e+07  1.771038e+06   

       ...            V59           V60           V61           V62  \
count  ...   40724.000000  1.804600e+04  51012.000000  50081.000000   
mean   ...      23.851035  1.903060e+04      5.978964     22.025242   
std    ...    2065.093238  9.703082e+04     20.856127    507.081174   
min    ...  -28112.000000 -5.224400e+05  -1912.190000   -101.200000   
25%    ...       0.690000  7.914825e+03      0.000000      2.620000   
50%    ...       1.290000  1.341000e+04      0.000000      9.380000   
75%    ...       2.792500  2.208301e+04      6.030000     20.180000   
max    ...  337618.500000  1.236127e+07   3201.230000  71101.790000   

                V63  industry_agriculture  industry_construction  \
count  5.061100e+04          51407.000000           51407.000000   
mean   1.777396e+01              0.111308               0.129068   
std    5.094217e+03              0.314516               0.335278   
min   -1.068620e+06              0.000000               0.000000   
25%    7.015000e+00              0.000000               0.000000   
50%    4.499000e+01              0.000000               0.000000   
75%    6.991000e+01              0.000000               0.000000   
max    1.741000e+05              1.000000               1.000000   

       industry_manufacture  industry_retail  bankruptcy_label  
count          51407.000000     51407.000000      51407.000000  
mean               0.379053         0.380571          0.004883  
std                0.485156         0.485532          0.069706  
min                0.000000         0.000000          0.000000  
25%                0.000000         0.000000          0.000000  
50%                0.000000         0.000000          0.000000  
75%                1.000000         1.000000          0.000000  
max                1.000000         1.000000          1.000000  

[8 rows x 68 columns]

In [4]:

def build_and_evaluate_model(df: pd.DataFrame) -> tuple:
    """Build and evaluate an XGBoost classifier on the provided DataFrame.

    Performs median imputation on missing values, splits the data using stratified
    sampling to maintain class imbalance, computes the appropriate scale_pos_weight,
    and validates the model using stratified cross-validation. Also tunes the decision
    threshold based on the F1 score.

    Args:
        df (pd.DataFrame): DataFrame containing feature columns and a target column
                           'bankruptcy_label'.

    Returns:
        tuple: (X_test with industry info, y_test, y_pred_proba from the test set)
    """
    X = df.drop(columns=["bankruptcy_label"])
    y = df["bankruptcy_label"]

    negatives = (y == 0).sum()
    positives = (y == 1).sum()
    scale_pos_weight = negatives / positives

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42
    )

    pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("classifier", xgb.XGBClassifier(
            scale_pos_weight=scale_pos_weight,
            eval_metric="auc",
            random_state=42,
            n_jobs=-1
        ))
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(
        pipeline, X_train, y_train, cv=cv, scoring="roc_auc", n_jobs=-1
    )

    pipeline.fit(X_train, y_train)

    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    y_pred = pipeline.predict(X_test)
    test_auc = roc_auc_score(y_test, y_pred_proba)

    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    f1_scores = 2 * precision * recall / (precision + recall + 1e-8)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5
    y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)

    print("Cross-validation AUC scores:", cv_scores)
    print("Mean CV AUC score:", cv_scores.mean())
    print("Test AUC score:", test_auc)
    print("Optimal threshold based on F1 score:", optimal_threshold)
    print("Confusion Matrix (default threshold):\n", confusion_matrix(y_test, y_pred))
    print("Classification Report (default threshold):\n", classification_report(y_test, y_pred))
    print("Confusion Matrix (optimal threshold):\n", confusion_matrix(y_test, y_pred_optimal))
    print("Classification Report (optimal threshold):\n", classification_report(y_test, y_pred_optimal))

    return X_test, y_test, y_pred_proba


def evaluate_by_industry(X_test: pd.DataFrame, y_test: pd.Series, y_pred_proba: np.ndarray,
                         threshold: float = 0.5) -> None:
    """Evaluate and print model performance metrics per industry.

    Derives an 'industry' column from one-hot encoded industry columns and groups the
    test set by industry. For each industry, it prints the support, ROC AUC, confusion
    matrix, and classification report.

    Args:
        X_test (pd.DataFrame): Test features DataFrame including industry one-hot columns.
        y_test (pd.Series): True labels for the test set.
        y_pred_proba (np.ndarray): Predicted probabilities for the positive class.
        threshold (float): Decision threshold for converting probabilities to class labels.
    """
    # Determine industry label from one-hot columns.
    def get_industry(row: pd.Series) -> str:
        if row.get("industry_agriculture", 0) == 1:
            return "agriculture"
        if row.get("industry_construction", 0) == 1:
            return "construction"
        if row.get("industry_manufacture", 0) == 1:
            return "manufacture"
        if row.get("industry_retail", 0) == 1:
            return "retail"
        return "unknown"

    industries = X_test.apply(get_industry, axis=1)
    # Compute predicted labels based on the chosen threshold.
    y_pred = (y_pred_proba >= threshold).astype(int)
    eval_df = pd.DataFrame({
        "industry": industries,
        "y_true": y_test.values,
        "y_pred": y_pred,
        "y_pred_proba": y_pred_proba
    })

    for industry, group in eval_df.groupby("industry"):
        print(f"\n--- Industry: {industry} ---")
        print("Support:", len(group))
        try:
            industry_auc = roc_auc_score(group["y_true"], group["y_pred_proba"])
        except Exception as err:
            industry_auc = None
        print("ROC AUC:", industry_auc)
        print("Confusion Matrix:")
        print(confusion_matrix(group["y_true"], group["y_pred"]))
        print("Classification Report:")
        print(classification_report(group["y_true"], group["y_pred"]))


X_test, y_test, y_pred_proba = build_and_evaluate_model(combined_df)
evaluate_by_industry(X_test, y_test, y_pred_proba, threshold=0.5)


Cross-validation AUC scores: [0.94479841 0.97442883 0.96500305 0.91250153 0.92382221]

Mean CV AUC score: 0.9441108075215846

Test AUC score: 0.9340852228303361

Optimal threshold based on F1 score: 0.5769484

Confusion Matrix (default threshold):
 [[10221    11]
 [   31    19]]

...

ROC AUC: 0.8872756604154063

Confusion Matrix:

[[3857    0]
 [   7    2]]

Classification Report:

precision    recall  f1-score   support

           0       1.00      1.00      1.00      3857
           1       1.00      0.22      0.36         9

    accuracy                           1.00      3866
   macro avg       1.00      0.61      0.68      3866
weighted avg       1.00      1.00      1.00      3866